---
title: "PROYECTO DE ANÁLISIS DE SUPERVIVENCIA"
subtitle: "Estimación del tiempo de supervivencia de vehículos nuevos vendidos por una concesionaria, hasta que requieren su primer mantenimiento"
date: today
author: "Byron Vinueza"
toc: true
toc-own-page: true
toc-title: "On this page"
categories:
    - Machine Learning
    - Data Science
    - Survival Analysis
format:
  html:
    html-math-method: katex
    self-contained: true
    code-fold: true
execute:
  warning: false
jupyter: python3
---



![](img/image.png){.center}

## Business Understanding (Entendimiento del negocio/problema)

* El problema central es la necesidad de estimar el tiempo de supervivencia de los vehículos nuevos vendidos por la concesionaria hasta que requieren su primer mantenimiento en el taller (aproximadamente a los 5000 kilómetros). Esto implica analizar el tiempo transcurrido desde la facturación/entrega del vehículo hasta la fecha de la primera visita al taller.

* La concesionaria busca identificar los factores que influyen en la duración de este tiempo de supervivencia, con el objetivo de mejorar la calidad del servicio postventa, y así incrementar la satisfacción de los clientes.

* Se espera que este análisis ayude al area de Marketing planificar estrategias y campañas de fidelización de clientes.

## Data Understanding (Entendimiento de los datos)

* La concencionaria cuenta con un sistema ERP que registra la información de las ventas de vehículos, y el historial de mantenimientos realizados en el taller, la información se encuentra en una base de datos oracle 11g. 

* Se cuenta con un datawarehouse donde la información se procesa mediante procesos ELT y se almacena en una base de datos SingleStore. 

* las variables mas relevantes son: 
    - id_vehiculo: identificador único del vehículo
    - fecha_factura: fecha de facturación del vehículo
    - modelo: modelo del vehículo
    - familia: familia del modelo 
    - fecha de mantenimiento 5000 kilómetros: fecha de la primera visita al taller
    - kilometraje: kilometraje del vehículo en la primera visita al taller

## Data Preparation (Preparación de los datos)

* La arquitectura de datos se conforma de la siguiente manera:
    - Base de datos Oracle 11g: se extrae la información de las ventas de vehículos y mantenimientos
    - Procesos ETL: se realizan transformaciones y limpieza de los datos
    - Base de datos SingleStore: se almacena la información procesada

![](img/arquitectura_data.png){.center}

### Procedimientos almacenados

### Dimensiones

* DIM_EMPRESA

```sql 
CREATE TABLE `DIM_EMPRESA` (
  `SK_EMPRESA` bigint(11) NOT NULL AUTO_INCREMENT,
  `COD_EMPRESA` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `EMPRESA` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FECHA_CARGA` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT 'Fecha en la que se realizo la carga' ,
  KEY `COD_EMPRESA` (`COD_EMPRESA`) USING CLUSTERED COLUMNSTORE,
  SHARD KEY `SK_EMPRESA` (`SK_EMPRESA`)
) ;

CREATE OR REPLACE PROCEDURE `CRG_DIM_EMPRESA`(VAR_SCHEMA varchar(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL) RETURNS void AS
DECLARE
	VAR_SQL VARCHAR(4000);
	VAR_ERROR VARCHAR(4000);
	VAR_DT_INICIO TIMESTAMP = CURRENT_TIMESTAMP;
BEGIN
START TRANSACTION;
	INSERT INTO CTR.LOG_PROCESO (MODULO, ETAPA,DT_FECHA_INICIO) VALUES ('DIMENSION',CONCAT('INICIO ',VAR_SCHEMA,'.CRG_DIM_EMPRESA'),VAR_DT_INICIO);
	COMMIT;
	
	VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP1_EMPRESA');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- GUARDA EN UNA TABLA TEMPORAL LOS REGISTROS ACTUALIZADOS
	VAR_SQL=CONCAT('CREATE TABLE ',VAR_SCHEMA,'.TMP1_EMPRESA (KEY (SK_EMPRESA) USING CLUSTERED COLUMNSTORE)
	SELECT  R.SK_EMPRESA,R.COD_EMPRESA 
	,IF(NVL(P.NOMBRE,R.EMPRESA)=R.EMPRESA,R.EMPRESA,P.NOMBRE) EMPRESA
	,IF(NVL(P.NOMBRE,R.EMPRESA)=R.EMPRESA
	,R.FECHA_CARGA,CURRENT_TIMESTAMP) FECHA_CARGA
	FROM DWH.DIM_EMPRESA R
	LEFT JOIN ',VAR_SCHEMA,'.S3S_ARCGMC P ON R.COD_EMPRESA=P.NO_CIA	
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- ELIMINA LOS DATOS DE LA DIM
	VAR_SQL=CONCAT('TRUNCATE TABLE DWH.DIM_EMPRESA');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- INSERTA LOS DATOS DE LA TMP1 A LA DIM
	VAR_SQL=CONCAT('	INSERT INTO DWH.DIM_EMPRESA
	(SK_EMPRESA,COD_EMPRESA, EMPRESA,FECHA_CARGA)
	SELECT A.SK_EMPRESA,A.COD_EMPRESA, A.EMPRESA,A.FECHA_CARGA
	FROM ',VAR_SCHEMA,'.TMP1_EMPRESA A	
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;
	
	VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP1_EMPRESA');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- INSERTA NUEVOS REGISTROS A LA DIM 
	VAR_SQL=CONCAT('INSERT INTO DWH.DIM_EMPRESA
	(COD_EMPRESA, EMPRESA)
	SELECT A.NO_CIA,A.NOMBRE FROM ',VAR_SCHEMA,'.S3S_ARCGMC A
	LEFT JOIN DWH.DIM_EMPRESA B ON A.NO_CIA=B.COD_EMPRESA
	WHERE B.COD_EMPRESA IS NULL
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

END;

```

* DIM_TIPO_DOCUMENTO

```sql
CREATE TABLE `DIM_TIPO_DOCUMENTO` (
  `SK_TIPO_DOCUMENTO` bigint(11) NOT NULL AUTO_INCREMENT,
  `COD_EMPRESA` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `COD_TIPO_DOCUMENTO` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `TIPO_DOCUMENTO` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FECHA_CARGA` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT 'Fecha en la que se realizo la carga' ,
  KEY `COD_EMPRESA` (`COD_EMPRESA`,`COD_TIPO_DOCUMENTO`) USING CLUSTERED COLUMNSTORE,
  SHARD KEY `SK_TIPO_DOCUMENTO` (`SK_TIPO_DOCUMENTO`)
) ;

CREATE OR REPLACE PROCEDURE `CRG_DIM_TIPO_DOCUMENTO`(VAR_SCHEMA varchar(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL) RETURNS void AS
DECLARE
	VAR_SQL VARCHAR(4000);
	VAR_ERROR VARCHAR(4000);
	VAR_DT_INICIO TIMESTAMP = CURRENT_TIMESTAMP;
BEGIN
START TRANSACTION;
	INSERT INTO CTR.LOG_PROCESO (MODULO, ETAPA,DT_FECHA_INICIO) VALUES ('DIMENSION',CONCAT('INICIO ',VAR_SCHEMA,'.CRG_DIM_TIPO_DOCUMENTO'),VAR_DT_INICIO);
	COMMIT;
	

	-- GUARDA EN UNA TABLA TEMPORAL LOS REGISTROS ACTUALIZADOS
	VAR_SQL=CONCAT('
       UPDATE DWH.DIM_TIPO_DOCUMENTO  R
		INNER JOIN ',VAR_SCHEMA,'.S3S_FAC_DOCUMENTOS  P ON R.COD_EMPRESA=P.NO_CIA and R.COD_TIPO_DOCUMENTO=P.TIPO
		SET R.COD_TIPO_DOCUMENTO=P.TIPO,
		R.TIPO_DOCUMENTO=P.DESCRIPCION;	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;



	-- INSERTA NUEVOS REGISTROS A LA DIM 
	VAR_SQL=CONCAT('INSERT INTO DWH.DIM_TIPO_DOCUMENTO
	(COD_EMPRESA,COD_TIPO_DOCUMENTO,TIPO_DOCUMENTO)
	SELECT A.NO_CIA, A.TIPO, A.DESCRIPCION FROM ',VAR_SCHEMA,'.S3S_FAC_DOCUMENTOS A
	LEFT JOIN DWH.DIM_TIPO_DOCUMENTO B ON A.NO_CIA=B.COD_EMPRESA AND A.TIPO = B.COD_TIPO_DOCUMENTO
	WHERE B.COD_EMPRESA IS NULL;
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

END;

```

* DIM_VENDEDOR

```sql
CREATE TABLE `DIM_VENDEDOR` (
  `SK_VENDEDOR` bigint(11) NOT NULL AUTO_INCREMENT,
  `COD_EMPRESA` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `COD_VENDEDOR` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `COD_USUARIO` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `NOMBRE_VENDEDOR` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FECHA_CARGA` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT 'Fecha en la que se realizo la carga' ,
  `ESTADO` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  KEY `COD_EMPRESA` (`COD_EMPRESA`,`COD_VENDEDOR`,`COD_USUARIO`) USING CLUSTERED COLUMNSTORE,
  SHARD KEY `SK_VENDEDOR` (`SK_VENDEDOR`)
) ;

CREATE OR REPLACE PROCEDURE `CRG_DIM_VENDEDOR`(VAR_SCHEMA varchar(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL) RETURNS void AS
DECLARE
	VAR_SQL VARCHAR(4000);
	VAR_ERROR VARCHAR(4000);
	VAR_DT_INICIO TIMESTAMP = CURRENT_TIMESTAMP;
BEGIN
START TRANSACTION;
	INSERT INTO CTR.LOG_PROCESO (MODULO, ETAPA,DT_FECHA_INICIO) VALUES ('DIMENSION',CONCAT('INICIO ',VAR_SCHEMA,'.CRG_DIM_VENDEDOR'),VAR_DT_INICIO);
	COMMIT;
	
	VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP1_VENDEDOR');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- GUARDA EN UNA TABLA TEMPORAL LOS REGISTROS ACTUALIZADOS
	VAR_SQL=CONCAT('CREATE TABLE ',VAR_SCHEMA,'.TMP1_VENDEDOR (KEY (SK_VENDEDOR) USING CLUSTERED COLUMNSTORE)
			SELECT  R.SK_VENDEDOR,R.COD_EMPRESA,      R.COD_VENDEDOR, 
    IF(NVL(P.SVVEUSUA  ,R.COD_USUARIO)=R.COD_USUARIO, R.COD_USUARIO , P.SVVEUSUA) COD_USUARIO   
	,IF(NVL(P.SVVENOMB,R.NOMBRE_VENDEDOR)=R.NOMBRE_VENDEDOR,R.NOMBRE_VENDEDOR,P.SVVENOMB) NOMBRE_VENDEDOR
	,IF(NVL(P.SVVENOMB,R.NOMBRE_VENDEDOR)=R.NOMBRE_VENDEDOR,R.FECHA_CARGA,CURRENT_TIMESTAMP) FECHA_CARGA
	,NVL(IF(NVL(O.ESTADO, R.ESTADO)=R.ESTADO, R.ESTADO, O.ESTADO), "NO DEFINIDO") ESTADO
	FROM DWH.DIM_VENDEDOR R
	LEFT JOIN ' ,VAR_SCHEMA,'.S3S_SVFVEND P ON R.COD_EMPRESA=P.NO_CIA and R.COD_VENDEDOR=P.SVVECODI
    LEFT JOIN ' ,VAR_SCHEMA,'.S3S_NOM_EMPLEADOS O ON O.NO_CIA = P.NO_CIA  AND O.NO_EMPLE =P.SVVECODI 

	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- ELIMINA LOS DATOS DE LA DIM
	VAR_SQL=CONCAT('TRUNCATE TABLE DWH.DIM_VENDEDOR');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

-- INSERTA LOS DATOS DE LA TMP1 A LA DIM
	VAR_SQL=CONCAT('INSERT INTO DWH.DIM_VENDEDOR
	(SK_VENDEDOR,COD_EMPRESA,COD_VENDEDOR,COD_USUARIO,NOMBRE_VENDEDOR,FECHA_CARGA, ESTADO)
	SELECT SK_VENDEDOR,COD_EMPRESA,COD_VENDEDOR,COD_USUARIO,NOMBRE_VENDEDOR,FECHA_CARGA, ESTADO
	FROM ' ,VAR_SCHEMA,'.TMP1_VENDEDOR ;
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;
	
	VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP1_VENDEDOR');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- INSERTA NUEVOS REGISTROS A LA DIM 
	VAR_SQL=CONCAT(' INSERT INTO DWH.DIM_VENDEDOR
	 (COD_EMPRESA,COD_VENDEDOR,COD_USUARIO,NOMBRE_VENDEDOR, ESTADO)
	SELECT A.NO_CIA, A.SVVECODI, A.SVVEUSUA, A.SVVENOMB , A.ESTADO
			FROM ( SELECT A.NO_CIA , A.SVVECODI  , A.SVVEUSUA , A.SVVENOMB  , NVL(B.ESTADO,"NO DEFINIDO") ESTADO
FROM ',VAR_SCHEMA,'.S3S_SVFVEND A 
LEFT JOIN ',VAR_SCHEMA,'.S3S_NOM_EMPLEADOS B ON A.NO_CIA = B.NO_CIA AND B.NO_EMPLE = A.SVVECODI 
						) A
	LEFT JOIN DWH.DIM_VENDEDOR B ON A.NO_CIA=B.COD_EMPRESA AND A.SVVECODI= B.COD_VENDEDOR
	WHERE B.COD_EMPRESA IS NULL;

	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

END;

```
* DIM_TIPO_FINANCIACION

```sql
CREATE TABLE `DIM_TIPO_FINANCIACION` (
  `SK_TIPO_FINANCIACION` bigint(11) NOT NULL AUTO_INCREMENT,
  `COD_EMPRESA` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `COD_TIPO_FINANCIACION` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `TIPO_FINANCIACION` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FECHA_CARGA` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT 'Fecha en la que se realizo la carga' ,
  `GRUPO_ENTIDAD` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  KEY `COD_EMPRESA` (`COD_EMPRESA`,`COD_TIPO_FINANCIACION`) USING CLUSTERED COLUMNSTORE,
  SHARD KEY `SK_TIPO_FINANCIACION` (`SK_TIPO_FINANCIACION`)
) ;

CREATE OR REPLACE PROCEDURE `CRG_DIM_TIPO_FINANCIACION`(VAR_SCHEMA varchar(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL) RETURNS void AS
DECLARE
	VAR_SQL VARCHAR(4000);
	VAR_ERROR VARCHAR(4000);
	VAR_DT_INICIO TIMESTAMP = CURRENT_TIMESTAMP;
BEGIN
START TRANSACTION;
	INSERT INTO CTR.LOG_PROCESO (MODULO, ETAPA,DT_FECHA_INICIO) VALUES ('DIMENSION',CONCAT('INICIO ',VAR_SCHEMA,'.CRG_DIM_TIPO_FINANCIACION'),VAR_DT_INICIO);
	COMMIT;
	
	VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP1_TIPO_FINANCIACION');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- GUARDA EN UNA TABLA TEMPORAL LOS REGISTROS ACTUALIZADOS
	VAR_SQL=CONCAT('CREATE TABLE ',VAR_SCHEMA,'.TMP1_TIPO_FINANCIACION (KEY (SK_TIPO_FINANCIACION) USING CLUSTERED COLUMNSTORE)
		SELECT  R.SK_TIPO_FINANCIACION,R.COD_EMPRESA, R.COD_TIPO_FINANCIACION 
	,IF(NVL(P.ENTIDAD,R.TIPO_FINANCIACION)=R.TIPO_FINANCIACION,R.TIPO_FINANCIACION,P.ENTIDAD) TIPO_FINANCIACION
	,IF(NVL(P.ENTIDAD,R.TIPO_FINANCIACION)=R.TIPO_FINANCIACION,R.FECHA_CARGA,CURRENT_TIMESTAMP) FECHA_CARGA
	,NVL(IF(NVL(P.GRUPO_ENTIDAD,R.GRUPO_ENTIDAD)=R.GRUPO_ENTIDAD,R.GRUPO_ENTIDAD,P.GRUPO_ENTIDAD),"NO DEFINIDO") GRUPO_ENTIDAD
	FROM DWH.DIM_TIPO_FINANCIACION R
	LEFT JOIN DSA.S3S_ARCCTF P ON R.COD_EMPRESA=P.NO_CIA and R.COD_TIPO_FINANCIACION=P.TIPO AND NVL(P.ESTADO,"X") <>"I"
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- ELIMINA LOS DATOS DE LA DIM
	VAR_SQL=CONCAT('TRUNCATE TABLE DWH.DIM_TIPO_FINANCIACION');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

-- INSERTA LOS DATOS DE LA TMP1 A LA DIM
	VAR_SQL=CONCAT('INSERT INTO DWH.DIM_TIPO_FINANCIACION
	(SK_TIPO_FINANCIACION,COD_EMPRESA,COD_TIPO_FINANCIACION,TIPO_FINANCIACION,FECHA_CARGA, GRUPO_ENTIDAD)
	SELECT SK_TIPO_FINANCIACION,COD_EMPRESA,COD_TIPO_FINANCIACION,TIPO_FINANCIACION,FECHA_CARGA, GRUPO_ENTIDAD
	FROM ' ,VAR_SCHEMA,'.TMP1_TIPO_FINANCIACION ;
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;
	
	VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP1_TIPO_FINANCIACION');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- INSERTA NUEVOS REGISTROS A LA DIM 
	VAR_SQL=CONCAT(' INSERT INTO DWH.DIM_TIPO_FINANCIACION
	 (COD_EMPRESA,COD_TIPO_FINANCIACION,TIPO_FINANCIACION,GRUPO_ENTIDAD)
	SELECT A.NO_CIA, A.TIPO, A.ENTIDAD, NVL(A.GRUPO_ENTIDAD, "NO DEFINIDO") FROM DSA.S3S_ARCCTF A
	LEFT JOIN DWH.DIM_TIPO_FINANCIACION B ON A.NO_CIA=B.COD_EMPRESA AND A.TIPO = B.COD_TIPO_FINANCIACION
	WHERE B.COD_EMPRESA IS NULL AND  NVL(A.ESTADO,"X") <>"I" ;

	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

END;


```

* DIM_FINANCIACION_VH

```sql

CREATE TABLE `DIM_FINANCIACION_VH` (
  `SK_FINANCIACION` bigint(11) NOT NULL AUTO_INCREMENT,
  `COD_FINANCIACION` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FINANCIACION` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FECHA_CARGA` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT 'Fecha en la que se realizo la carga' ,
  KEY `COD_FINANCIACION` (`COD_FINANCIACION`) USING CLUSTERED COLUMNSTORE,
  SHARD KEY `SK_FINANCIACION` (`SK_FINANCIACION`)
);

INSERT INTO DIM_FINANCIACION_VH (COD_FINANCIACION,FINANCIACION) VALUES
	 ('-1','NO DEFINIDO'),
	 ('CA','CUOTA DE ALCANCE'),
	 ('CD','CREDITO DIRECTO'),
	 ('CT','CREDITO TRADICIONAL'),
	 ('TSN','TSN'),
	 ('CON','CONTADO');
   
```
* DIM_AGENCIA

```sql
CREATE TABLE `DIM_AGENCIA` (
  `SK_AGENCIA` bigint(11) NOT NULL AUTO_INCREMENT,
  `COD_EMPRESA` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `COD_AGENCIA` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `AGENCIA` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `PAIS` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `PROVINCIA` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `CIUDAD` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FECHA_CARGA` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT 'Fecha en la que se realizo la carga' ,
  KEY `COD_EMPRESA` (`COD_EMPRESA`,`COD_AGENCIA`) USING CLUSTERED COLUMNSTORE,
  SHARD KEY `SK_AGENCIA` (`SK_AGENCIA`)
) ;

CREATE OR REPLACE PROCEDURE `CRG_DIM_AGENCIA`(VAR_SCHEMA varchar(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL) RETURNS void AS
DECLARE
	VAR_SQL VARCHAR(4000);
	VAR_ERROR VARCHAR(4000);
	VAR_DT_INICIO TIMESTAMP = CURRENT_TIMESTAMP;
BEGIN
START TRANSACTION;
	INSERT INTO CTR.LOG_PROCESO (MODULO, ETAPA,DT_FECHA_INICIO) VALUES ('DIMENSION',CONCAT('INICIO ',VAR_SCHEMA,'.CRG_DIM_AGENCIA'),VAR_DT_INICIO);
	COMMIT;
	


	-- ACTUALIZA LOS REGISTROS EN LA DIM 

	VAR_SQL=CONCAT('
		UPDATE DWH.DIM_AGENCIA  R
		INNER JOIN 
		(SELECT A.NO_CIA, A. CODIGO, A.NOMBRE AS AGENCIA, DECODE(B.PAIS,01,"ECUADOR","NO DEFINIDO") AS PAIS, NVL(C.DESCRIPCION,"NO DEFINIDO") AS PROVINCIA, 
				NVL(B.DESCRIPCION,"NO DEFINIDO") AS CIUDAD
				FROM ',VAR_SCHEMA,'.S3S_AGENCIAS A
				LEFT JOIN ',VAR_SCHEMA,'.S3S_ARCCCAN B ON A.CIUDAD = B.CANTON AND A.PROVINCIA = B.PROVINCIA
				LEFT JOIN ',VAR_SCHEMA,'.S3S_ARCCPRO C ON B.PROVINCIA = C.PROVINCIA)  P ON R.COD_EMPRESA=P.NO_CIA AND R.COD_AGENCIA = P.CODIGO
		SET R.AGENCIA=P.AGENCIA,
			R.PAIS=P.PAIS,
			R.PROVINCIA=P.PROVINCIA,
			R.CIUDAD=P.CIUDAD
		;');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

		-- INSERTA NUEVOS REGISTROS A LA DIM 
	VAR_SQL=CONCAT('INSERT INTO DWH.DIM_AGENCIA
		(COD_EMPRESA, COD_AGENCIA, AGENCIA, PAIS, PROVINCIA, CIUDAD)
		SELECT A.NO_CIA,A.CODIGO,A.AGENCIA,A.PAIS,A.PROVINCIA,A.CIUDAD
		FROM
		(SELECT A.NO_CIA, A. CODIGO, A.NOMBRE AS AGENCIA, DECODE(B.PAIS,01,"ECUADOR","NO DEFINIDO") AS PAIS, NVL(C.DESCRIPCION,"NO DEFINIDO") AS PROVINCIA, 
				NVL(B.DESCRIPCION,"NO DEFINIDO") AS CIUDAD
				FROM ',VAR_SCHEMA,'.S3S_AGENCIAS A
				LEFT JOIN ',VAR_SCHEMA,'.S3S_ARCCCAN B ON A.CIUDAD = B.CANTON AND A.PROVINCIA = B.PROVINCIA
				LEFT JOIN ',VAR_SCHEMA,'.S3S_ARCCPRO C ON B.PROVINCIA = C.PROVINCIA) A
		LEFT JOIN DWH.DIM_AGENCIA B ON A.NO_CIA=B.COD_EMPRESA AND A.CODIGO = B.COD_AGENCIA
		WHERE B.COD_EMPRESA IS NULL 
		');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

END;

```
* DIM_LINEA_NEGOCIO

```sql
CREATE TABLE `DIM_LINEA_NEGOCIO` (
  `SK_LINEA_NEGOCIO` bigint(11) NOT NULL AUTO_INCREMENT,
  `COD_EMPRESA` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `COD_LINEA_NEGOCIO` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `LINEA_NEGOCIO` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FECHA_CARGA` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT 'Fecha en la que se realizo la carga' ,
  KEY `COD_EMPRESA` (`COD_EMPRESA`,`COD_LINEA_NEGOCIO`) USING CLUSTERED COLUMNSTORE,
  SHARD KEY `SK_LINEA_NEGOCIO` (`SK_LINEA_NEGOCIO`)
) ;

CREATE OR REPLACE PROCEDURE `CRG_DIM_LINEA_NEGOCIO`(VAR_SCHEMA varchar(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL) RETURNS void AS
DECLARE
	VAR_SQL VARCHAR(4000);
	VAR_ERROR VARCHAR(4000);
	VAR_DT_INICIO TIMESTAMP = CURRENT_TIMESTAMP;
BEGIN
START TRANSACTION;
	INSERT INTO CTR.LOG_PROCESO (MODULO, ETAPA,DT_FECHA_INICIO) VALUES ('DIMENSION',CONCAT('INICIO ',VAR_SCHEMA,'.CRG_DIM_LINEA_NEGOCIO'),VAR_DT_INICIO);
	COMMIT;
	
	VAR_SQL=CONCAT('
			UPDATE DWH.DIM_LINEA_NEGOCIO  R
			INNER JOIN (SELECT   A.NO_CIA, A.NO_LINEA, A.DESCRIPCION
										FROM ',VAR_SCHEMA,'.S3S_ARCJLI A
										UNION 
										SELECT COD_EMPRESA, S3S_ID, NAME 
										FROM ',VAR_SCHEMA,'.CRM_CB_LINEANEGOCIO WHERE COD_EMPRESA = "01" AND s3s_id = "ce")  P ON R.COD_EMPRESA=P.NO_CIA AND R.COD_LINEA_NEGOCIO = P.NO_LINEA
			SET R.LINEA_NEGOCIO =P.DESCRIPCION ;');
		 EXECUTE IMMEDIATE VAR_SQL;
		COMMIT;


	-- INSERTA NUEVOS REGISTROS A LA DIM 
	VAR_SQL=CONCAT('INSERT INTO DWH.DIM_LINEA_NEGOCIO
	(COD_EMPRESA, COD_LINEA_NEGOCIO, LINEA_NEGOCIO)
	SELECT A.NO_CIA, A.NO_LINEA, A.DESCRIPCION FROM (SELECT   A.NO_CIA, A.NO_LINEA, A.DESCRIPCION
																								FROM ',VAR_SCHEMA,'.S3S_ARCJLI A
																								UNION 
																								SELECT COD_EMPRESA, S3S_ID, NAME 
																								FROM ',VAR_SCHEMA,'.CRM_CB_LINEANEGOCIO WHERE COD_EMPRESA = "01" AND s3s_id = "ce") A
	LEFT JOIN DWH.DIM_LINEA_NEGOCIO B ON A.NO_CIA=B.COD_EMPRESA AND A.NO_LINEA = B.COD_LINEA_NEGOCIO
	WHERE B.COD_EMPRESA IS NULL
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

END;

```

* DIM_VEHICULO

```sql
CREATE TABLE `DIM_VEHICULO` (
  `SK_VEHICULO` bigint(11) NOT NULL AUTO_INCREMENT,
  `COD_EMPRESA` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `COD_LINEA_NEGOCIO` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `CHASIS` varchar(30) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `PLACA` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `MARCA` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `ANIO_VEHICULO` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `SVMAMASTER` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `SFX` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `MODELO` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FECHA_CARGA` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT 'Fecha en la que se realizo la carga' ,
  `TIPO_VEHICULO` varchar(200) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `COLOR` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `CLASE` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FAMILIA` varchar(30) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  KEY `SK_VEHICULO` (`COD_EMPRESA`,`COD_LINEA_NEGOCIO`,`CHASIS`) USING CLUSTERED COLUMNSTORE,
  SHARD KEY `SK_VEHICULO_2` (`SK_VEHICULO`)
) ;

CREATE OR REPLACE PROCEDURE `CRG_DIM_VEHICULO`(VAR_SCHEMA varchar(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL) RETURNS void AS
DECLARE
	VAR_SQL VARCHAR(4000);
	VAR_ERROR VARCHAR(4000);
	VAR_DT_INICIO TIMESTAMP = CURRENT_TIMESTAMP;
BEGIN
START TRANSACTION;
	INSERT INTO CTR.LOG_PROCESO (MODULO, ETAPA,DT_FECHA_INICIO) VALUES ('DIMENSION',CONCAT('INICIO ',VAR_SCHEMA,'.CRG_DIM_VEHICULO'),VAR_DT_INICIO);
	COMMIT;
	

VAR_SQL=CONCAT('UPDATE DWH.DIM_VEHICULO R
				LEFT JOIN ' ,VAR_SCHEMA,'.S3S_CAT_VEHICULO P ON R.COD_EMPRESA = P.COD_EMPRESA AND R.COD_LINEA_NEGOCIO = P.COD_LINEA_NEGOCIO AND R.CHASIS = P.CHASIS 
		SET 	R.FECHA_CARGA = IF(NVL(P.PLACA,R.PLACA)=R.PLACA,R.FECHA_CARGA,CURRENT_TIMESTAMP)
				, R.PLACA = NVL(IF(NVL(P.PLACA,R.PLACA)=R.PLACA,R.PLACA,P.PLACA), "NO DEFINIDO")  
				, R.MARCA = NVL(IF(NVL(P.MARCA,R.MARCA)=R.MARCA,R.MARCA,P.MARCA), "NO DEFINIDO")  
				, R.ANIO_VEHICULO = NVL(IF(NVL(P.ANIO_VEHICULO,R.ANIO_VEHICULO)=R.ANIO_VEHICULO,R.ANIO_VEHICULO,P.ANIO_VEHICULO), "NO DEFINIDO")  
				, R.SVMAMASTER = NVL(IF(NVL(P.SVMAMASTER,R.SVMAMASTER)=R.SVMAMASTER,R.SVMAMASTER,P.SVMAMASTER), "NO DEFINIDO") 
				, R.SFX = NVL(IF(NVL(P.SFX,R.SFX)=R.SFX,R.SFX,P.SFX), "NO DEFINIDO") 
				, R.MODELO = NVL(IF(NVL(P.MODELO,R.MODELO)=R.MODELO,R.MODELO,P.MODELO), "NO DEFINIDO")  
				, R.TIPO_VEHICULO = NVL(IF(NVL(P.TIPO_VEHICULO,R.TIPO_VEHICULO)=R.TIPO_VEHICULO,R.TIPO_VEHICULO,P.TIPO_VEHICULO), "NO DEFINIDO")  
				, R.COLOR = NVL(IF(NVL(P.COLOR,R.COLOR)=R.COLOR,R.COLOR,P.COLOR), "NO DEFINIDO")  
				, R.CLASE = NVL(IF(NVL(P.CLASE,R.CLASE)=R.CLASE,R.CLASE,P.CLASE), "NO DEFINIDO") 
				, R.FAMILIA = NVL(IF(NVL(P.FAMILIA,R.FAMILIA)=R.FAMILIA,R.FAMILIA,P.FAMILIA), "NO DEFINIDO") 
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

		-- INSERTA NUEVOS REGISTROS A LA DIM 
	VAR_SQL=CONCAT('INSERT INTO DWH.DIM_VEHICULO
		( COD_EMPRESA, COD_LINEA_NEGOCIO, CHASIS, PLACA, MARCA,ANIO_VEHICULO,SVMAMASTER ,SFX, MODELO, TIPO_VEHICULO, COLOR, CLASE,FAMILIA)
		SELECT  A.COD_EMPRESA, A.COD_LINEA_NEGOCIO, A.CHASIS, NVL(A.PLACA,"NO DEFINIDO"), NVL(A.MARCA,"NO DEFINIDO"), 
		NVL(A.ANIO_VEHICULO,"NO DEFINIDO"), NVL(A.SVMAMASTER,"NO DEFINIDO"),NVL(A.SFX,"NO DEFINIDO"), NVL(A.MODELO, "NO DEFINIDO"),
		NVL(A.TIPO_VEHICULO, "NO DEFINIDO"), NVL(A.COLOR, "NO DEFINIDO"), NVL(A.CLASE, "NO DEFINIDO") , NVL(A.FAMILIA, "NO DEFINIDO") 
		FROM DSA.S3S_CAT_VEHICULO A
		LEFT JOIN DWH.DIM_VEHICULO B ON A.COD_LINEA_NEGOCIO = B.COD_LINEA_NEGOCIO AND A.COD_EMPRESA = B.COD_EMPRESA AND A.CHASIS = B.CHASIS
		WHERE B.COD_EMPRESA IS NULL
		');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

END;

```

* DIM_TECNICO_ASESOR

```sql

CREATE TABLE `DIM_TECNICO_ASESOR` (
  `SK_TECNICO_ASESOR` bigint(11) NOT NULL AUTO_INCREMENT,
  `COD_EMPRESA` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `COD_TECNICO_ASESOR` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `TECNICO_ASESOR` varchar(100) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `FECHA_CARGA` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT 'Fecha en la que se realizo la carga' ,
  `ESTADO` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  KEY `COD_EMPRESA` (`COD_EMPRESA`,`COD_TECNICO_ASESOR`) USING CLUSTERED COLUMNSTORE,
  SHARD KEY `SK_TECNICO_ASESOR` (`SK_TECNICO_ASESOR`)
) ;

CREATE OR REPLACE PROCEDURE `CRG_DIM_TECNICO_ASESOR`(VAR_SCHEMA varchar(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL) RETURNS void AS
DECLARE
	VAR_SQL VARCHAR(4000);
	VAR_ERROR VARCHAR(4000);
	VAR_DT_INICIO TIMESTAMP = CURRENT_TIMESTAMP;
BEGIN
START TRANSACTION;
	INSERT INTO CTR.LOG_PROCESO (MODULO, ETAPA,DT_FECHA_INICIO) VALUES ('DIMENSION',CONCAT('INICIO ',VAR_SCHEMA,'.CRG_DIM_TECNICO_ASESOR'),VAR_DT_INICIO);
	COMMIT;
	
	VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP1_TECNICO_ASESOR');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- GUARDA EN UNA TABLA TEMPORAL LOS REGISTROS ACTUALIZADOS
	VAR_SQL=CONCAT('CREATE TABLE ',VAR_SCHEMA,'.TMP1_TECNICO_ASESOR (KEY (SK_TECNICO_ASESOR) USING CLUSTERED COLUMNSTORE)
	SELECT  R.SK_TECNICO_ASESOR,R.COD_EMPRESA, R.COD_TECNICO_ASESOR 
	,IF(NVL(P.NOMBRE,R.TECNICO_ASESOR)=R.TECNICO_ASESOR,R.TECNICO_ASESOR,P.NOMBRE) TECNICO_ASESOR
	,IF(NVL(P.NOMBRE,R.TECNICO_ASESOR)=R.TECNICO_ASESOR,R.FECHA_CARGA,CURRENT_TIMESTAMP) FECHA_CARGA
	,NVL(IF(NVL(P.ESTADO,R.ESTADO)=R.ESTADO, R.ESTADO, P.ESTADO), "NO DEFINIDO") ESTADO
	FROM DWH.DIM_TECNICO_ASESOR R
	LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_TECNIASESOR P ON R.COD_EMPRESA=P.NO_CIA AND R.COD_TECNICO_ASESOR = P.TECNIASESOR_ID
	 ;
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- ELIMINA LOS DATOS DE LA DIM
	VAR_SQL=CONCAT('TRUNCATE TABLE DWH.DIM_TECNICO_ASESOR');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- INSERTA LOS DATOS DE LA TMP1 A LA DIM
	VAR_SQL=CONCAT('INSERT INTO DWH.DIM_TECNICO_ASESOR
	(SK_TECNICO_ASESOR,COD_EMPRESA, COD_TECNICO_ASESOR,TECNICO_ASESOR,FECHA_CARGA, ESTADO)
	SELECT A.SK_TECNICO_ASESOR,A.COD_EMPRESA, A.COD_TECNICO_ASESOR, A.TECNICO_ASESOR,A.FECHA_CARGA, ESTADO
	FROM ',VAR_SCHEMA,'.TMP1_TECNICO_ASESOR A	
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;
	
	VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP1_TECNICO_ASESOR');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

	-- INSERTA NUEVOS REGISTROS A LA DIM 
	VAR_SQL=CONCAT('INSERT INTO DWH.DIM_TECNICO_ASESOR
	(COD_EMPRESA, COD_TECNICO_ASESOR, TECNICO_ASESOR, ESTADO)
	SELECT A.NO_CIA, A.TECNIASESOR_ID, A.NOMBRE, NVL(A.ESTADO, "NO DEFINIDO") ESTADO 
FROM ',VAR_SCHEMA,'.S3S_SE_TECNIASESOR A
	LEFT JOIN DWH.DIM_TECNICO_ASESOR B ON A.NO_CIA=B.COD_EMPRESA AND A.TECNIASESOR_ID = B.COD_TECNICO_ASESOR
	WHERE B.COD_EMPRESA IS NULL
	');
	 EXECUTE IMMEDIATE VAR_SQL;
	COMMIT;

END;


```

### Tabla de Hechos

* FC_VENTA_VS_MANTENIMIENTO

```sql
CREATE TABLE `FC_VENTA_VS_MANTENIMIENTO` (
  `SK_EMPRESA` bigint(20) DEFAULT 1,
  `SK_FECHA` bigint(20) DEFAULT 19700101,
  `SK_FACTURA` text CHARACTER SET utf8 COLLATE utf8_general_ci,
  `NO_FISICO` text CHARACTER SET utf8 COLLATE utf8_general_ci,
  `SK_TIPO_DOCUMENTO` bigint(11) DEFAULT 11,
  `SK_VENDEDOR` bigint(11) DEFAULT NULL,
  `SK_TIPO_FINANCIACION` bigint(11) DEFAULT 1,
  `SK_AGENCIA` bigint(11) DEFAULT 1,
  `SK_AGENCIA_FACTURA` bigint(11) DEFAULT 1,
  `SK_LINEA_NEGOCIO` bigint(11) DEFAULT 1,
  `SK_FECHA_SALIDA` bigint(20) DEFAULT 19700101,
  `SK_VEHICULO` bigint(11) DEFAULT 1,
  `SK_AGENCIA_MNT1` bigint(11) DEFAULT 1,
  `SK_TECNICO_ASESOR1` bigint(11) DEFAULT 1,
  `MANTENIMIENTO_1000` decimal(18,0) DEFAULT NULL,
  `SK_FECHA_MANTENIMIENTO_1000` bigint(20) DEFAULT 19700101,
  `SK_AGENCIA_MNT5` bigint(11) DEFAULT 1,
  `SK_TECNICO_ASESOR5` bigint(11) DEFAULT 1,
  `MANTENIMIENTO_5000` decimal(18,0) DEFAULT NULL,
  `SK_FECHA_MANTENIMIENTO_5000` bigint(20) DEFAULT 19700101,
  `SK_AGENCIA_MNT10` bigint(11) DEFAULT 1,
  `SK_TECNICO_ASESOR10` bigint(11) DEFAULT 1,
  `MANTENIMIENTO_10000` decimal(18,0) DEFAULT NULL,
  `SK_FECHA_MANTENIMIENTO_10000` bigint(20) DEFAULT 19700101,
  `SK_FINANCIACION` bigint(20) DEFAULT NULL,
  KEY `FC_VENTAS_NUEVOS_EXONERADOS_MANTENIMIENTOS` (`SK_FECHA`,`SK_FACTURA`,`SK_VEHICULO`) USING CLUSTERED COLUMNSTORE
  , SHARD KEY () 
) ;

CREATE OR REPLACE PROCEDURE `CRG_FC_VENTA_VS_MANTENIMIENTO`(VAR_SCHEMA varchar(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL, VAR_EMPRESA varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci NULL, VAR_FECHA_INICIO date NULL, VAR_FECHA_FIN date NULL, VAR_LINEA_NEGOCIO_NUEVOS varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci NULL, VAR_LINEA_NEGOCIO_EXONERADOS varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci NULL, VAR_LINEA_NEGOCIO_MNT varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci NULL) RETURNS void AS
DECLARE
	VAR_SQL VARCHAR(8000);
	VAR_ERROR VARCHAR(4000);
	VAR_DT_INICIO TIMESTAMP = CURRENT_TIMESTAMP;
BEGIN
START TRANSACTION;
INSERT INTO CTR.LOG_PROCESO (MODULO, ETAPA,DT_FECHA_INICIO) VALUES ('TABLA DE HECHOS',CONCAT('INICIO ',VAR_SCHEMA,'.CRG_FC_VENTA_VS_MANTENIMIENTO'),VAR_DT_INICIO);
COMMIT;

VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP_VENTAS_NUEVOS_EXONERADOS');
EXECUTE IMMEDIATE VAR_SQL;
COMMIT;

VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP_VENTA_MANTENIMIENTO');
EXECUTE IMMEDIATE VAR_SQL;
COMMIT;

/*Descripción: Creación de la tabla temporal de las ventas efectivas en las líneas de negocios nuevos y exonerados exluyendo a los clientes 
* 	CARLOS LARREA Y COMERCIAL HIDROVO*/

VAR_SQL=CONCAT(' CREATE TABLE DSA.TMP_VENTAS_NUEVOS_EXONERADOS (KEY (NO_FACTU,CHASIS) USING CLUSTERED COLUMNSTORE)
                    SELECT
                            AA.FECHA , AA.NO_FACTU , AA.NO_FISICO , AA.TIPO_DOC , AA.NO_VENDEDOR
                            , BB.CHASIS
                            , AA.LINEA , AA.NO_PEDIDO
                            , NVL(AA.CENTRO_ORIGEN,AA.CENTROD) COD_AGENCIA
                            , AA.CENTROD COD_AGENCIA_FACTURA
                            , AA.NO_CIA COD_EMPRESA
                            , AA.LINEA_NEGOCIO
                            , AA.FECHA_SALIDA
                            , CASE WHEN (AA.LINEA IN ("CY", "C8", "C1", "C2", "C3", "C4", "C5", "C6", "C7", "C9", "D1")) THEN "TSN"
                                        WHEN ((I.SALDO_FINANCIAR = 0) AND (I.CUOTA_DE_ALCANCE = 0)) THEN "CON"
                                        WHEN ((I.SALDO_FINANCIAR > 0) AND (AA.LINEA  = "CB")) THEN "CD"
                                        WHEN ((I.SALDO_FINANCIAR = 0) AND (I.CUOTA_DE_ALCANCE > 0) AND (AA.LINEA  = "CB")) THEN "CA"
                                        WHEN (AA.LINEA  = "SV") THEN "CA"
                                        WHEN (((I.CUOTA_DE_ALCANCE > 0) OR (I.SALDO_FINANCIAR= 0)) AND (AA.LINEA  <> "CB")) THEN "CT"
                                        WHEN (AA.LINEA  = "VE") THEN "CD"
                            ELSE "CT" END AS COD_FINANCIACION
                    FROM  
                            ',VAR_SCHEMA,'.S3S_FAC_VENTAS_CAB AA
                            INNER JOIN ',VAR_SCHEMA,'.S3S_FAC_VENTAS_VEHICULOS BB ON AA.NO_CIA =BB.NO_CIA AND AA.RUTA =BB.RUTA AND AA.TIPO_DOC =BB.TIPO_DOC AND AA.NO_FACTU =BB.NO_FACTU
                            INNER JOIN ',VAR_SCHEMA,'.S3S_SVFCOTI I ON  AA.CENTROD = I.CENTRO AND AA.NO_PEDIDO = I.NUMERO_COTIZACION AND I.SVLICODI = AA.LINEA_NEGOCIO  AND AA.NO_CIA = I.NO_CIA
                    WHERE 
                            AA.NO_CIA = "',VAR_EMPRESA,'"
                            AND AA.LINEA_NEGOCIO = "',VAR_LINEA_NEGOCIO_NUEVOS,'"
                            AND NVL(AA.IND_ANU_DEV,"X")<>"A"
                            AND DATE(AA.FECHA) BETWEEN "',VAR_FECHA_INICIO,'" AND "',VAR_FECHA_FIN,'"  
                            AND AA.NO_CLIENTE NOT IN (69157,186117,20781,23437,63323)
                ');
EXECUTE IMMEDIATE VAR_SQL;
COMMIT;

VAR_SQL=CONCAT('INSERT INTO ',VAR_SCHEMA,'.TMP_VENTAS_NUEVOS_EXONERADOS
                    SELECT
                            AA.FECHA
                            , AA.NO_FACTU
                            , AA.NO_FISICO
                            , AA.TIPO_DOC , AA.NO_VENDEDOR
                            , BB.CHASIS
                            , AA.LINEA , AA.NO_PEDIDO
                            , NVL(AA.CENTRO_ORIGEN,AA.CENTROD) COD_AGENCIA
                            , AA.CENTROD COD_AGENCIA_FACTURA
                            , AA.NO_CIA COD_EMPRESA
                            , AA.LINEA_NEGOCIO
                            , AA.FECHA_SALIDA
                            , CASE WHEN (AA.LINEA IN ("CY", "C8", "C1", "C2", "C3", "C4", "C5", "C6", "C7", "C9", "D1")) THEN "TSN"
                                        WHEN ((I.SALDO_FINANCIAR = 0) AND (I.CUOTA_DE_ALCANCE = 0)) THEN "CON"
                                        WHEN ((I.SALDO_FINANCIAR > 0) AND (AA.LINEA  = "CB")) THEN "CD"
                                        WHEN ((I.SALDO_FINANCIAR = 0) AND (I.CUOTA_DE_ALCANCE > 0) AND (AA.LINEA  = "CB")) THEN "CA"
                                        WHEN (AA.LINEA  = "SV") THEN "CA"
                                        WHEN (((I.CUOTA_DE_ALCANCE > 0) OR (I.SALDO_FINANCIAR= 0)) AND (AA.LINEA  <> "CB")) THEN "CT"
                                        WHEN (AA.LINEA  = "VE") THEN "CD"
                                ELSE "CT" END AS COD_FINANCIACION
                    FROM 
                            ',VAR_SCHEMA,'.S3S_FAC_VENTAS_CAB AA
                            INNER JOIN ',VAR_SCHEMA,'.S3S_FAC_VENTAS_VEHICULOS BB ON AA.NO_CIA =BB.NO_CIA AND BB.RUTA =AA.RUTA AND BB.TIPO_DOC =AA.TIPO_DOC AND BB.NO_FACTU =AA.NO_FACTU
                            INNER JOIN ',VAR_SCHEMA,'.S3S_SVFCOTI I ON  AA.CENTROD = I.CENTRO AND AA.NO_PEDIDO = I.NUMERO_COTIZACION AND I.SVLICODI = AA.LINEA_NEGOCIO  AND AA.NO_CIA = I.NO_CIA
                    WHERE 
                            AA.NO_CIA = "',VAR_EMPRESA,'"
                            AND AA.LINEA_NEGOCIO = "',VAR_LINEA_NEGOCIO_EXONERADOS,'"
                            AND NVL(AA.IND_ANU_DEV,"X")<> "A"
                            AND AA.TIPO_DOC  IN ("EX","NE")
                            AND DATE(AA.FECHA) BETWEEN "',VAR_FECHA_INICIO,'"   AND "',VAR_FECHA_FIN,'" 
                            AND AA.NO_CLIENTE NOT IN (69157,186117,20781,23437,63323) 
                    ');
EXECUTE IMMEDIATE VAR_SQL;
COMMIT;

-- Descripción: Creación de la tabla temporal de las ordenes creadas de los vehículos que ingresan al taller a mantenimiento de los  1000, 5000, 10000 km 

VAR_SQL=CONCAT('CREATE TABLE ',VAR_SCHEMA,'.TMP_VENTA_MANTENIMIENTO (KEY (CHASIS) USING CLUSTERED COLUMNSTORE)
                    SELECT 
                            T.*
                            , T1.CODIGO AS AGENCIA_1 
                            , T1.COD_TECNICO_ASESOR AS COD_TECNICO_ASESOR1
                            , T1.KM_RELACIONADO AS K1
                            , T1.FECHA_FACTURA FECHA_FACTURA_K1
                            , T2.CODIGO AS AGENCIA_5
                            , T2.COD_TECNICO_ASESOR AS COD_TECNICO_ASESOR5
                            , T2.KM_RELACIONADO AS K5
                            , T2.FECHA_FACTURA FECHA_FACTURA_K5
                            , T3.CODIGO AS AGENCIA_10
                            , T3.COD_TECNICO_ASESOR AS COD_TECNICO_ASESOR10
                            , T3.KM_RELACIONADO AS K10
                            , T3.FECHA_FACTURA FECHA_FACTURA_K10
                    FROM 
                            ',VAR_SCHEMA,'.TMP_VENTAS_NUEVOS_EXONERADOS  T
                            LEFT JOIN (
                                                    SELECT
                                                            B.CHASIS_NO  , A.KILOMETRAJE, A.CODIGO
                                                            , A.TECNIASESOR_ID COD_TECNICO_ASESOR
                                                            , (D.KM_RELACIONADO) KM_RELACIONADO
                                                            , A.FECHA_CIERRE FECHA_FACTURA
                                                    FROM 
                                                            ',VAR_SCHEMA,'.S3S_SE_ORDENREP A
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_VEHICULOS B ON A.NO_CIA = B.NO_CIA AND  A.NUMCBS = B.NUMCBS
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_TRABORDEN C ON A.NO_CIA = C.NO_CIA AND A.CODIGO = C.CODIGO AND A.ORDENREP_ID = C.ORDENREP_ID
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_MENUSERV3 D ON C.NO_CIA = D.NO_CIA AND C.CODSERV1 = D.CODSERV1 AND C.CODSERV2 = D.CODSERV2 AND C.CODSERV3 = D.CODSERV3
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_ORDENREP_NC NC ON NC.NO_CIA = A.NO_CIA AND A.ORDENREP_ID = NC.ORDENREP_ID_ORIGEN
                                                    WHERE 
                                                            A.NO_CIA = "',VAR_EMPRESA,'"
                                                            AND A.LINEA_NEGOCIO = "',VAR_LINEA_NEGOCIO_MNT,'"
                                                            AND A.ESTADO_ID in ("85","90")
                                                            AND D.KM_RELACIONADO = 1000
                                                            AND NC.ORDENREP_ID_ORIGEN IS NULL
                                                ) T1  ON T1.CHASIS_NO = T.CHASIS AND YEAR(T1.FECHA_FACTURA) >= YEAR(T.FECHA)
                            LEFT JOIN (
                                                    SELECT
                                                            B.CHASIS_NO , A.KILOMETRAJE, A.CODIGO
                                                            , A.TECNIASESOR_ID COD_TECNICO_ASESOR
                                                            , NVL(A.FECHA_FACTURA,A.FECHA_CIERRE) FECHA_FACTURA
                                                            , (D.KM_RELACIONADO) KM_RELACIONADO
                                                    FROM 
                                                            ',VAR_SCHEMA,'.S3S_SE_ORDENREP A
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_VEHICULOS B ON A.NO_CIA = B.NO_CIA AND  A.NUMCBS = B.NUMCBS
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_TRABORDEN C ON A.NO_CIA = C.NO_CIA AND A.CODIGO = C.CODIGO AND A.ORDENREP_ID = C.ORDENREP_ID
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_MENUSERV3 D ON C.NO_CIA = D.NO_CIA AND C.CODSERV1 = D.CODSERV1 AND C.CODSERV2 = D.CODSERV2 AND C.CODSERV3 = D.CODSERV3
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_ORDENREP_NC NC ON NC.NO_CIA = A.NO_CIA AND A.ORDENREP_ID = NC.ORDENREP_ID_ORIGEN
                                                    WHERE 
                                                            A.NO_CIA = "',VAR_EMPRESA,'"
                                                            AND A.LINEA_NEGOCIO = "',VAR_LINEA_NEGOCIO_MNT,'"
                                                            AND A.ESTADO_ID in ("85","90")
                                                            AND D.KM_RELACIONADO = 5000
                                                            AND NC.ORDENREP_ID_ORIGEN IS NULL
                                                ) T2  ON T2.CHASIS_NO = T.CHASIS
                            LEFT JOIN (
                                                    SELECT
                                                            B.CHASIS_NO , A.KILOMETRAJE, A.CODIGO   
                                                            , A.TECNIASESOR_ID COD_TECNICO_ASESOR
                                                            , NVL(A.FECHA_FACTURA,A.FECHA_CIERRE) FECHA_FACTURA
                                                            , (D.KM_RELACIONADO) KM_RELACIONADO
                                                    FROM 
                                                            ',VAR_SCHEMA,'.S3S_SE_ORDENREP A
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_VEHICULOS B ON A.NO_CIA = B.NO_CIA AND  A.NUMCBS = B.NUMCBS
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_TRABORDEN C ON A.NO_CIA = C.NO_CIA AND A.CODIGO = C.CODIGO AND A.ORDENREP_ID = C.ORDENREP_ID
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_MENUSERV3 D ON C.NO_CIA = D.NO_CIA AND C.CODSERV1 = D.CODSERV1 AND C.CODSERV2 = D.CODSERV2 AND C.CODSERV3 = D.CODSERV3
                                                            LEFT JOIN ',VAR_SCHEMA,'.S3S_SE_ORDENREP_NC NC ON NC.NO_CIA = A.NO_CIA AND A.ORDENREP_ID = NC.ORDENREP_ID_ORIGEN
                                                    WHERE 
                                                            A.NO_CIA = "',VAR_EMPRESA,'"
                                                            AND A.LINEA_NEGOCIO = "',VAR_LINEA_NEGOCIO_MNT,'"
                                                            AND A.ESTADO_ID in ("85","90")
                                                            AND D.KM_RELACIONADO = 10000
                                                            AND NC.ORDENREP_ID_ORIGEN IS NULL
                                                ) T3  ON T3.CHASIS_NO = T.CHASIS 
                    ');
EXECUTE IMMEDIATE VAR_SQL;
COMMIT;	

VAR_SQL=CONCAT('DELETE FROM DWH.FC_VENTA_VS_MANTENIMIENTO A 
                    WHERE A.SK_EMPRESA = (SELECT SK_EMPRESA FROM DWH.DIM_EMPRESA WHERE COD_EMPRESA = "',VAR_EMPRESA,'")
                    AND SK_FECHA >= (SELECT SK_FECHA FROM DWH.DIM_FECHA WHERE FECHA = "',VAR_FECHA_INICIO,'") 
                    AND SK_FECHA <= (SELECT SK_FECHA FROM DWH.DIM_FECHA WHERE FECHA = "',VAR_FECHA_FIN,'") 
                    ');
EXECUTE IMMEDIATE VAR_SQL;			
COMMIT;

/*Descripción: Inserción de registros en la tabla de hechos */

VAR_SQL=CONCAT('INSERT INTO DWH.FC_VENTA_VS_MANTENIMIENTO 
                    SELECT 
                            EM.SK_EMPRESA
                            , NVL(FF.SK_FECHA,19700101) SK_FECHA
                            , AA.NO_FACTU
                            , AA.NO_FISICO
                            , TD.SK_TIPO_DOCUMENTO
                            , DV.SK_VENDEDOR 
                            , TF.SK_TIPO_FINANCIACION 
                            , NVL(AG1.SK_AGENCIA,1) SK_AGENCIA
                            , NVL(AG2.SK_AGENCIA,1) SK_AGENCIA_FACTURA
                            , NVL(LI.SK_LINEA_NEGOCIO,1) SK_LINEA_NEGOCIO
                            , NVL(FF3.SK_FECHA ,19700101) SK_FECHA_SALIDA
                            , NVL(VEH.SK_VEHICULO,1) SK_VEHICULO
                            , NVL(A1.SK_AGENCIA,1) SK_AGENCIA_MNT1
                            , NVL(TA1.SK_TECNICO_ASESOR,1) SK_TECNICO_ASESOR1
                            , NVL(AA.K1,-1) AS MANTENIMIENTO_1000
                            , NVL(FK1.SK_FECHA,19700101) AS SK_FECHA_MANTENIMIENTO_1000
                            , NVL(A5.SK_AGENCIA,1) SK_AGENCIA_MNT5
                            , NVL(TA5.SK_TECNICO_ASESOR,1) SK_TECNICO_ASESOR5
                            , NVL(AA.K5,-1) AS MANTENIMIENTO_5000
                            , NVL(FK5.SK_FECHA,19700101)  AS SK_FECHA_MANTENIMIENTO_5000
                            , NVL(A10.SK_AGENCIA,1) SK_AGENCIA_MNT10
                            , NVL(TA10.SK_TECNICO_ASESOR,1) SK_TECNICO_ASESOR10
                            , NVL(AA.K10,-1) AS MANTENIMIENTO_10000
                            , NVL(FK10.SK_FECHA,19700101)  AS SK_FECHA_MANTENIMIENTO_10000
                            , NVL(FV.SK_FINANCIACION,1 ) SK_FINANCIACION
                    FROM 
                            DSA.TMP_VENTA_MANTENIMIENTO AA
                            LEFT JOIN DWH.DIM_EMPRESA EM ON EM.COD_EMPRESA=AA.COD_EMPRESA
                            LEFT JOIN DWH.DIM_FECHA FF ON DATE(FF.FECHA)=DATE(AA.FECHA)
                            LEFT JOIN DWH.DIM_TIPO_DOCUMENTO TD ON TD.COD_TIPO_DOCUMENTO=AA.TIPO_DOC AND  TD.COD_EMPRESA =AA.COD_EMPRESA
                            LEFT JOIN DWH.DIM_VENDEDOR DV ON DV.COD_VENDEDOR =AA.NO_VENDEDOR AND DV.COD_EMPRESA=AA.COD_EMPRESA
                            LEFT JOIN DWH.DIM_TIPO_FINANCIACION TF  ON TF.COD_TIPO_FINANCIACION =AA.LINEA AND TF.COD_EMPRESA =AA.COD_EMPRESA
                            LEFT JOIN DWH.DIM_FINANCIACION_VH FV ON FV.COD_FINANCIACION = AA.COD_FINANCIACION
                            LEFT JOIN DWH.DIM_AGENCIA AG1 ON AG1.COD_EMPRESA =AA.COD_EMPRESA AND AG1.COD_AGENCIA =AA.COD_AGENCIA
                            LEFT JOIN DWH.DIM_AGENCIA AG2 ON AG2.COD_EMPRESA =AA.COD_EMPRESA AND AG2.COD_AGENCIA =AA.COD_AGENCIA_FACTURA
                            LEFT JOIN DWH.DIM_LINEA_NEGOCIO LI ON LI.COD_EMPRESA =AA.COD_EMPRESA AND LI.COD_LINEA_NEGOCIO =AA.LINEA_NEGOCIO 
                            LEFT JOIN DWH.DIM_FECHA FF3 ON  DATE(FF3.FECHA) = DATE(AA.FECHA_SALIDA)
                            LEFT JOIN DWH.DIM_VEHICULO VEH ON VEH.CHASIS=AA.CHASIS AND VEH.COD_EMPRESA = AA.COD_EMPRESA AND VEH.COD_LINEA_NEGOCIO =AA.LINEA_NEGOCIO
                            LEFT JOIN DWH.DIM_FECHA FK1 ON DATE(FK1.FECHA) = DATE(AA.FECHA_FACTURA_K1) 
                            LEFT JOIN DWH.DIM_FECHA FK5 ON DATE(FK5.FECHA) = DATE(AA.FECHA_FACTURA_K5) 
                            LEFT JOIN DWH.DIM_FECHA FK10 ON DATE(FK10.FECHA) = DATE(AA.FECHA_FACTURA_K10) 
                            LEFT JOIN DWH.DIM_AGENCIA A1 ON AA.COD_EMPRESA = A1.COD_EMPRESA AND AA.AGENCIA_1 = A1.COD_AGENCIA 
                            LEFT JOIN DWH.DIM_AGENCIA A5 ON AA.COD_EMPRESA = A5.COD_EMPRESA AND AA.AGENCIA_5 = A5.COD_AGENCIA 
                            LEFT JOIN DWH.DIM_AGENCIA A10 ON AA.COD_EMPRESA = A10.COD_EMPRESA AND AA.AGENCIA_10 = A10.COD_AGENCIA 
                            LEFT JOIN DWH.DIM_TECNICO_ASESOR TA1 ON AA.COD_EMPRESA = TA1.COD_EMPRESA AND AA.COD_TECNICO_ASESOR1 = TA1.COD_TECNICO_ASESOR 
                            LEFT JOIN DWH.DIM_TECNICO_ASESOR TA5 ON AA.COD_EMPRESA = TA5.COD_EMPRESA AND AA.COD_TECNICO_ASESOR5 = TA5.COD_TECNICO_ASESOR 
                            LEFT JOIN DWH.DIM_TECNICO_ASESOR TA10 ON AA.COD_EMPRESA = TA10.COD_EMPRESA AND AA.COD_TECNICO_ASESOR10 = TA10.COD_TECNICO_ASESOR 
                ');
EXECUTE IMMEDIATE VAR_SQL;
COMMIT;	

VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP_VENTAS_NUEVOS_EXONERADOS');
    EXECUTE IMMEDIATE VAR_SQL;
COMMIT;

VAR_SQL=CONCAT('DROP TABLE IF EXISTS ',VAR_SCHEMA,'.TMP_VENTA_MANTENIMIENTO');
    EXECUTE IMMEDIATE VAR_SQL;
COMMIT;
        
END;

```

### Vista 

* VW_VENTA_VS_MANTENIMIENTO

```sql
CREATE  VIEW `VW_VENTA_VS_MANTENIMIENTO` AS
SELECT
	`B`.`EMPRESA` AS `EMPRESA`,
	DATE(`C`.`FECHA`) AS `FECHA`,
	`D`.`TIPO_DOCUMENTO` AS `TIPO_DOCUMENTO`,
	`A`.`SK_FACTURA` AS `FACTURA`,
	`E`.`NOMBRE_VENDEDOR` AS `NOMBRE_VENDEDOR`,
	`F`.`TIPO_FINANCIACION` AS `TIPO_FINANCIACION`,
	`G`.`FINANCIACION` AS `FINANCIACION`,
	`H`.`AGENCIA` AS `AGENCIA`,
	`I`.`AGENCIA` AS `AGENCIA_FACTURA`,
	`J`.`LINEA_NEGOCIO` AS `LINEA_NEGOCIO`,
	DATE(`K`.`FECHA`) AS `FECHA_SALIDA`,
	`L`.`CHASIS` AS `CHASIS`,
	`L`.`FAMILIA` AS `FAMILIA_VH`,
	`L`.`MODELO` AS `MODELO`,
	`A`.`MANTENIMIENTO_1000` AS `MANTENIMIENTO_1000`,
	DATE(`M`.`FECHA`) AS `FECHA_MNT_1000`,
	`P`.`AGENCIA` AS `AGENCIA_MNT_1000`,
	`S`.`TECNICO_ASESOR` AS `TECNICO_ASESOR_MNT_1000`,
	`A`.`MANTENIMIENTO_5000` AS `MANTENIMIENTO_5000`,
	DATE(`N`.`FECHA`) AS `FECHA_MNT_5000`,
	`Q`.`AGENCIA` AS `AGENCIA_MNT_5000`,
	`T`.`TECNICO_ASESOR` AS `TECNICO_ASESOR_MNT_5000`,
	`A`.`MANTENIMIENTO_10000` AS `MANTENIMIENTO_10000`,
	DATE(`O`.`FECHA`) AS `FECHA_MNT_10000`,
	`R`.`AGENCIA` AS `AGENCIA_MNT_10000`,
	`U`.`TECNICO_ASESOR` AS `TECNICO_ASESOR_MNT_10000`
FROM
	((((((((((((((((((((`FC_VENTA_VS_MANTENIMIENTO` as `A`
LEFT JOIN `DIM_EMPRESA` as `B` ON
	(`A`.`SK_EMPRESA` = `B`.`SK_EMPRESA`))
LEFT JOIN `DIM_FECHA` as `C` ON
	(`A`.`SK_FECHA` = `C`.`SK_FECHA`))
LEFT JOIN `DIM_TIPO_DOCUMENTO` as `D` ON
	(`A`.`SK_TIPO_DOCUMENTO` = `D`.`SK_TIPO_DOCUMENTO`))
LEFT JOIN `DIM_VENDEDOR` as `E` ON
	(`A`.`SK_VENDEDOR` = `E`.`SK_VENDEDOR`))
LEFT JOIN `DIM_TIPO_FINANCIACION` as `F` ON
	(`A`.`SK_TIPO_FINANCIACION` = `F`.`SK_TIPO_FINANCIACION`))
LEFT JOIN `DIM_FINANCIACION_VH` as `G` ON
	(`A`.`SK_FINANCIACION` = `G`.`SK_FINANCIACION`))
LEFT JOIN `DIM_AGENCIA` as `H` ON
	(`A`.`SK_AGENCIA` = `H`.`SK_AGENCIA`))
LEFT JOIN `DIM_AGENCIA` as `I` ON
	(`A`.`SK_AGENCIA_FACTURA` = `I`.`SK_AGENCIA`))
LEFT JOIN `DIM_LINEA_NEGOCIO` as `J` ON
	(`A`.`SK_LINEA_NEGOCIO` = `J`.`SK_LINEA_NEGOCIO`))
LEFT JOIN `DIM_FECHA` as `K` ON
	(`A`.`SK_FECHA_SALIDA` = `K`.`SK_FECHA`))
LEFT JOIN `DIM_VEHICULO` as `L` ON
	(`A`.`SK_VEHICULO` = `L`.`SK_VEHICULO`))
LEFT JOIN `DIM_FECHA` as `M` ON
	(`A`.`SK_FECHA_MANTENIMIENTO_1000` = `M`.`SK_FECHA`))
LEFT JOIN `DIM_FECHA` as `N` ON
	(`A`.`SK_FECHA_MANTENIMIENTO_5000` = `N`.`SK_FECHA`))
LEFT JOIN `DIM_FECHA` as `O` ON
	(`A`.`SK_FECHA_MANTENIMIENTO_10000` = `O`.`SK_FECHA`))
LEFT JOIN `DIM_AGENCIA` as `P` ON
	(`A`.`SK_AGENCIA_MNT1` = `P`.`SK_AGENCIA`))
LEFT JOIN `DIM_AGENCIA` as `Q` ON
	(`A`.`SK_AGENCIA_MNT5` = `Q`.`SK_AGENCIA`))
LEFT JOIN `DIM_AGENCIA` as `R` ON
	(`A`.`SK_AGENCIA_MNT10` = `R`.`SK_AGENCIA`))
LEFT JOIN `DIM_TECNICO_ASESOR` as `S` ON
	(`A`.`SK_TECNICO_ASESOR1` = `S`.`SK_TECNICO_ASESOR`))
LEFT JOIN `DIM_TECNICO_ASESOR` as `T` ON
	(`A`.`SK_TECNICO_ASESOR5` = `T`.`SK_TECNICO_ASESOR`))
LEFT JOIN `DIM_TECNICO_ASESOR` as `U` ON
	(`A`.`SK_TECNICO_ASESOR10` = `U`.`SK_TECNICO_ASESOR`)) ;


```

## Analisis exploratorio de datos

### Carga e Inspección Inicial de los Datos:


In [ ]:
#| label: read_data
#| warning: false

from ssconnect import connect_singlestore
import pandas as pd

engine = connect_singlestore()
sql="SELECT * FROM DWH.VW_VENTA_VS_MANTENIMIENTO "
df= pd.read_sql(sql, engine)

#Mostrar las primeras filas del dataset para tener una visión general.

df.head()

In [ ]:
#| label: data_info
#| warning: false

import datetime as dt
# Convertir la columna 'FECHA' a tipo datetime

# Verificar el tamaño del dataset (número de filas y columnas).
print(f"Dataset shape: {df.shape}")
# Número de registros por año 
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d', errors='coerce')
df['FECHA'].dt.year.value_counts().sort_index(ascending=False)

### Número de Ventas y Mantenimientos por Año

Desde la parte de vista comercial, las ventas se contabilizan como la suma de las facturas menos las notas de crédito o devolución.


In [ ]:
#| label: num_ventas_comercial
#| warning: false

import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Gráfico de barras, número de ventas por año, sumar facturas y restar notas de crédito 
df['AÑO'] = df['FECHA'].dt.year
df['CONT'] = df['TIPO_DOCUMENTO'].apply(lambda x: 1 if x == 'FACTURA CREDITO' else -1)

# Calcular el número total de ventas por año  y mes 
ventas_por_año = df.groupby('AÑO')['CONT'].sum()
# Crear el gráfico de linea 
fig = go.Figure()
fig.add_trace(go.Scatter(x=ventas_por_año.index, y=ventas_por_año.values, mode='lines+markers', name='Ventas Totales'))
fig.update_layout(title='Número de Ventas por Año', xaxis_title='Año', yaxis_title='Número de Ventas')
fig.show()

Para el presente análisis, se considera las ventas efectivas, es decir, las ventas que no han sido devueltas o anuladas. Para ello, se considera el campo fecha de salida, el cual indica la fecha en la que el vehículo salió de la agencia.


In [ ]:
#| label: num_ventas_efectivas
#| warning: false

# Gráfico de barras, número de ventas efectivas por año, filtrar por fecha de salida que debe ser mayor a 1970-01-01
df['FECHA_SALIDA'] = pd.to_datetime(df['FECHA_SALIDA'], format='%Y-%m-%d', errors='coerce')
df_vtas= df[df['FECHA_SALIDA'] > dt.datetime(1970, 1, 1)]

# Calcular el número total de ventas efectivas por año  y mes
ventas_efectivas_por_año = df_vtas.groupby('AÑO')['CONT'].sum()
# Crear el gráfico de linea
fig = go.Figure()
fig.add_trace(go.Scatter(x=ventas_efectivas_por_año.index, y=ventas_efectivas_por_año.values, mode='lines+markers', name='Ventas Efectivas'))
fig.update_layout(title='Número de Ventas Efectivas por Año', xaxis_title='Año', yaxis_title='Número de Ventas Efectivas')
fig.show()

Se considera como fecha pivot de analisis, la fecha de salida del vehículo. Se excluye los vehciulos que se han entregado en el año 2025.


In [ ]:
#| label: num_vehiculos_entrega
#| warning: false
# Gráfico de barras, número de vehículos entregados por año, filtrar por fecha de salida que debe ser mayor a 1970-01-01
df['FECHA_SALIDA'] = pd.to_datetime(df['FECHA_SALIDA'], format='%Y-%m-%d', errors='coerce')
df_vtas= df[df['FECHA_SALIDA'] > dt.datetime(1970, 1, 1)]
df_vtas= df_vtas[df_vtas['FECHA_SALIDA'].dt.year != 2025]
df_vtas['AÑO']= df_vtas['FECHA_SALIDA'].dt.year
# Calcular el número total de vehículos entregados por año  
vehiculos_entregados_por_año = df_vtas.groupby('AÑO')['CONT'].count()
# Crear el gráfico de linea
fig = go.Figure()
fig.add_trace(go.Scatter(x=vehiculos_entregados_por_año.index, y=vehiculos_entregados_por_año.values, mode='lines+markers', name='Vehículos Entregados'))
fig.update_layout(title='Número de Vehículos Entregados por Año', xaxis_title='Año', yaxis_title='Número de Vehículos Entregados')
fig.show()

#### Vechículos con mantenimiento de 1000 km


In [ ]:
#| label: num_vehiculos_mant_1000
#| warning: false


# Gráfico de barras, número de vehículos con mantenimiento de 1000 km por año, filtrar por fecha de salida que debe ser mayor a 1970-01-01
df['FECHA_SALIDA'] = pd.to_datetime(df['FECHA_SALIDA'], format='%Y-%m-%d', errors='coerce')
df_vtas= df[df['FECHA_SALIDA'] > dt.datetime(1970, 1, 1)]
df_vtas= df_vtas[df_vtas['FECHA_SALIDA'].dt.year != 2025]
df_vtas['AÑO']= df_vtas['FECHA_SALIDA'].dt.year
# Calcular el número total de vehículos con mantenimiento de 1000 km por año , columna MANTENIMIENTO_1000 mayor que 0

vehiculos_mant_1000_por_año = df_vtas[df_vtas['MANTENIMIENTO_1000'] > 0].groupby('AÑO')['MANTENIMIENTO_1000'].count()
# Crear el gráfico de linea
fig = go.Figure()
fig.add_trace(go.Scatter(x=vehiculos_mant_1000_por_año.index, y=vehiculos_mant_1000_por_año.values, mode='lines+markers', name='Vehículos con Mantenimiento 1000 km'))  
fig.update_layout(title='Número de Vehículos con Mantenimiento de 1000 km por Año', xaxis_title='Año', yaxis_title='Número de Vehículos con Mantenimiento 1000 km')
fig.show()

In [ ]:
#| label: porc_vehiculos_mant_1000
#| warning: false

# Gráfico de barras, porcentaje de vehículos con mantenimiento de 1000 km por año, filtrar por fecha de salida que debe ser mayor a 1970-01-01
df['FECHA_SALIDA'] = pd.to_datetime(df['FECHA_SALIDA'], format='%Y-%m-%d', errors='coerce')
df_vtas= df[df['FECHA_SALIDA'] > dt.datetime(1970, 1, 1)]
df_vtas= df_vtas[df_vtas['FECHA_SALIDA'].dt.year != 2025]
df_vtas['AÑO']= df_vtas['FECHA_SALIDA'].dt.year
# Calcular el número total de vehículos con mantenimiento de 1000 km por año , columna MANTENIMIENTO_1000 mayor que 0
vehiculos_mant_1000_por_año = df_vtas[df_vtas['MANTENIMIENTO_1000'] > 0].groupby('AÑO')['MANTENIMIENTO_1000'].count()
# Calcular el número total de vehículos entregados por año
vehiculos_entregados_por_año = df_vtas.groupby('AÑO')['CONT'].count()
# Calcular el porcentaje de vehículos con mantenimiento de 1000 km por año
porcentaje_vehiculos_mant_1000_por_año = (vehiculos_mant_1000_por_año / vehiculos_entregados_por_año) * 100
# Crear el gráfico de linea
fig = go.Figure()
fig.add_trace(go.Scatter(x=porcentaje_vehiculos_mant_1000_por_año.index, y=porcentaje_vehiculos_mant_1000_por_año.values, mode='lines+markers', name='Porcentaje de Vehículos con Mantenimiento 1000 km'))
fig.update_layout(title='Porcentaje de Vehículos con Mantenimiento de 1000 km por Año', xaxis_title='Año', yaxis_title='Porcentaje de Vehículos con Mantenimiento 1000 km')
fig.show()